In [1]:
import sys
import os

# append the path of the current notebook to the sys.path
cur_dir = os.path.abspath(os.path.curdir)
root_dir = os.path.abspath(os.path.join(cur_dir, '..'))
sys.path.append(root_dir)


In [2]:
import pandas as pd

In [8]:
from src.data.load_data import load_data
from src.features.engineering import engineer_features
from src.models.evaluate import evaluate_model
from src.data.split_data import split_data
from sklearn.ensemble import RandomForestClassifier
from src.models.RandomForest import train_RandomForest

In [16]:
from sklearn.ensemble import RandomForestClassifier

# load the data
data = engineer_features(load_data())

# get the feature matrix and target vector
X_train, X_test, y_train, y_test = split_data(data, 'readmitted')

# create a model
model = RandomForestClassifier(n_estimators=1, max_depth=1, min_samples_split=4, n_jobs=-1, random_state=14)

# fit the model
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = evaluate_model(model, X_train, X_test, y_train, y_test)

#feature importance
feature_importance = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

print(feature_importance)

/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)


Train accuracy:  0.8883991240384076
Test accuracy:  0.888404847690796
number_inpatient    1.0
time_in_hospital    0.0
diag_2_98           0.0
diag_2_92           0.0
diag_2_93           0.0
                   ... 
diag_1_16           0.0
diag_1_15           0.0
diag_1_14           0.0
diag_1_13           0.0
diabetesMed_Yes     0.0
Length: 527, dtype: float64


In [13]:
feature_importance.value_counts()

0.0    526
1.0      1
dtype: int64

In [26]:
accuracy

0.5391090730429086

In [9]:
data = engineer_features(load_data())#[cols])

# Split the data
X_train, X_test, y_train, y_test = split_data(data, 'readmitted', test_size=0.2)

# pick the best k features
#X_train, X_test = pick_best_k_features(X_train, X_test, y_train, y_test, k=50)

# Train the model
rf = train_RandomForest(X_train, y_train, n_estimators=1, max_depth=1, min_samples_split=4)

# Evaluate the model
evaluate_model(rf, X_train, X_test, y_train, y_test)

/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)


Train accuracy:  0.8883948312288115
Test accuracy:  0.8884248796305394


(0.8883948312288115, 0.8884248796305394)

In [7]:
data = engineer_features(load_data())#[cols])

# Split the data
X_train, X_test, y_train, y_test = split_data(data, 'readmitted', test_size=0.2)

# Create a model
model = RandomForestClassifier(n_estimators=100, max_depth=1, min_samples_split=1, n_jobs=-1)

# Fit the model
model.fit(X_train, y_train)

# Evaluate the model
evaluate_model(model, X_train, X_test, y_train, y_test)

/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)


InvalidParameterError: The 'min_samples_split' parameter of RandomForestClassifier must be an int in the range [2, inf) or a float in the range (0.0, 1.0]. Got 1 instead.

In [16]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Load the iris dataset as an example
df = load_data()
df = engineer_features(df)
X = df.drop(columns=['readmitted'])
y = df['readmitted']

# Select the top 2 features using chi-squared test
k_best = SelectKBest(score_func=chi2, k=10)
X_new = k_best.fit_transform(X, y)

# Print the selected features
selected_features = k_best.get_support(indices=True)
print("Selected features:", X.columns[selected_features])

Selected features: Index(['admission_source_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'diag_1_42'],
      dtype='object')


In [18]:
engineer_features(load_data())['readmitted'].value_counts()

/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
/Users/dumindu/Documents/Repositories/diabetes-ucidata/src/features/engineering.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = pd.get_dummies(data, columns=cat_cols, drop_first=True)


0    54864
1    46902
Name: readmitted, dtype: int64

Female             54708
Male               47055
Unknown/Invalid        3
Name: gender, dtype: int64

In [15]:
load_data()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1.0,MC,Pediatrics-Endocrinology,41.0,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),1,1,7,3.0,MC,InternalMedicine,59.0,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),1,1,7,2.0,MC,InternalMedicine,11.0,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),1,1,7,2.0,MC,InternalMedicine,44.0,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,1,7,1.0,MC,InternalMedicine,51.0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),1,3,7,3.0,MC,InternalMedicine,51.0,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,AfricanAmerican,Female,[80-90),1,4,5,5.0,MC,InternalMedicine,33.0,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,Caucasian,Male,[70-80),1,1,7,1.0,MC,InternalMedicine,53.0,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,Caucasian,Female,[80-90),2,3,7,10.0,MC,Surgery-General,45.0,...,No,Up,No,No,No,No,No,Ch,Yes,NO
